In [1]:
from IPython import get_ipython

from PIL import Image
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import time
import random
from math import ceil
from tqdm import tqdm
import os

import torch.utils.data as data
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torch.optim as optim
import torch.nn.functional as F


# 设置后续需要的参数（部分实际未被使用）
class VarsConfig(object):
    train_data = "train.csv"
    test_data = "val.csv"
    pre_data = "test.csv"

    epoch = 30
    batch_size = 64
    img_height = 64
    img_weight = 64
    seed = 666


config = VarsConfig()


# 设置图像变换
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机旋转变换
    transforms.RandomCrop(64, padding=4),  # 随机裁剪
    transforms.ToTensor(),  # 转换为tensor格式
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 标准化
])

test_transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为tensor格式
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 标准化
])



# 读取训练集和测试集
train_csv = pd.read_csv(config.train_data, header=None).values
test_csv = pd.read_csv(config.test_data, header=None).values
val_csv = pd.read_csv(config.pre_data, header=None).values

train_label = train_csv[0:65000, 0]
train_image = train_csv[0:65000, 1:]
test_label = test_csv[0:12500, 0]
test_image= test_csv[0:12500, 1:]


train_image = torch.from_numpy(train_image).type(torch.LongTensor)
test_image = torch.from_numpy(test_image).type(torch.LongTensor)
train_label = torch.from_numpy(train_label).type(torch.LongTensor)
test_label = torch.from_numpy(test_label).type(torch.LongTensor)

# 读取训练集和测试集
train_image = train_image.view(-1, 2, 64, 64).float()
test_image = test_image.view(-1, 2, 64, 64).float()
train_data = torch.utils.data.TensorDataset(train_image, train_label)
test_data = torch.utils.data.TensorDataset(test_image, test_label)
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=config.batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=config.batch_size,
                                          shuffle=False)



class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3,
                      stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3,
                      stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=12):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv0 = nn.Sequential(
            nn.Conv2d(2, 3, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(3),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)  # strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv0(x)
        out = self.conv1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():
    return ResNet(ResidualBlock)

# 定义模型保存函数
def save_models(epoch):
    torch.save(model, "Resnet_{}_RG.mdl".format(epoch + 1))
    print("Chekcpoint saved")



model = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)


# train the model
best_acc = 0.0
results = pd.DataFrame(columns=['epoch', 'train_acc', 'test_acc',
                                'train_loss', 'test_loss'], index=range(0, config.epoch))
for epoch in range(config.epoch):
    print("epoch:", +epoch)
    model.train()
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)

        lossvalue = criterion(output, labels)
        lossvalue.backward()
        optimizer.step()

    # evaluate the model
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    correct_num_test = 0
    train_loss = 0.0
    train_acc = 0.0
    correct_num_train = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            train_loss += criterion(output, labels).item()
            pred = output.max(1, keepdim=True)[1]
            correct_num_train += pred.eq(labels.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    correct_rate = 100. * correct_num_train / len(train_loader.dataset)
    train_acc = correct_num_train / len(train_loader.dataset)
    print('Train Accuracy: {0}/{1}({2:.4}%),Train Loss: {3}'.format(
        correct_num_train, len(train_loader.dataset), correct_rate, train_loss))

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            test_loss += criterion(output, labels).item()
            pred = output.max(1, keepdim=True)[1]
            correct_num_test += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_test / len(test_loader.dataset)
    test_acc = correct_num_test / len(test_loader.dataset)


    if test_acc > best_acc:
        save_models(epoch)
        best_acc = test_acc
        best_epoch = epoch + 1
    results.iloc[epoch] = [epoch + 1, train_acc,
                           test_acc, train_loss, test_loss]
    print('Test  Accuracy: {0}/{1}({2:.4}%), Test Loss: {3}'.format(
        correct_num_test, len(test_loader.dataset), correct_rate, test_loss))


# %%
results.to_csv('resnet_RG.csv', index=False)


  0%|          | 3/1016 [00:00<00:58, 17.44it/s]

ResNet(
  (conv0): Sequential(
    (0): Conv2d(2, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequen

100%|██████████| 1016/1016 [01:17<00:00, 13.09it/s]


Train Accuracy: 47005/65000(72.32%),Train Loss: 0.012887772414775995


/chpc/home/XYJia/miniconda3/envs/pytorch1.0/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/chpc/home/XYJia/miniconda3/envs/pytorch1.0/lib/python3.6/site-packages/torch/serialization.py:250: UserWarning: Couldn't retrieve source code for container of type ResidualBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
  0%|          | 3/1016 [00:00<00:54, 18.63it/s]

...

Train Accuracy: 64710/65000(99.55%),Train Loss: 0.00021694417046920324


  0%|          | 4/1016 [00:00<00:32, 31.10it/s]

Test  Accuracy: 9868/12500(78.94%), Test Loss: 0.022896438319683073
epoch: 29


100%|██████████| 1016/1016 [00:36<00:00, 27.69it/s]


Train Accuracy: 64489/65000(99.21%),Train Loss: 0.00037025984233030333
Test  Accuracy: 9698/12500(77.58%), Test Loss: 0.0249389559173584
